In [1]:
import os
import math
import glob
import requests
import numpy as np
import pandas as pd
#from tqdm import tqdm
from io import StringIO
from tqdm import tqdm_notebook as tqdm

In [2]:
molecule = []
iso_slug = []
isotopologue = []
path_mol_iso = []
for line in open('./data/url/api__urls.txt'):
    molecule.append(line.split('/')[-4])
    iso_slug.append(line.split('/')[-3])
    isotopologue.append(line.split('/')[-2])
    path_mol_iso.append(line.split('/')[-4] + '/' + line.split('/')[-3] + '/' + line.split('/')[-2])

molecule_list = list(set(molecule))
molecule_list.sort(key=molecule.index)

iso_slug_list = list(set(iso_slug))
iso_slug_list.sort(key=iso_slug.index)

isotopologue_list = list(set(isotopologue))
isotopologue_list.sort(key=isotopologue.index)

path_mol_iso_list = list(set(path_mol_iso))
path_mol_iso_list.sort(key=path_mol_iso.index)

print('Molecule:', molecule_list)
print('Iso-slug:', iso_slug_list)
print('Isotopologue:', isotopologue_list)
print('Total:', path_mol_iso_list)


Molecule: ['AlH', 'C2H2', 'C2', 'CO2', 'H2O', 'H3O_p', 'NH3', 'TiO']
Iso-slug: ['27Al-1H', '12C2-1H2', '12C2', '12C-16O2', '1H2-16O', '1H3-16O_p', '14N-1H3', '46Ti-16O', '47Ti-16O', '48Ti-16O', '49Ti-16O', '50Ti-16O']
Isotopologue: ['AlHambra', 'aCeTY', '8states', 'UCL-4000', 'POKAZATEL', 'eXeL', 'CoYuTe', 'Toto']
Total: ['AlH/27Al-1H/AlHambra', 'C2H2/12C2-1H2/aCeTY', 'C2/12C2/8states', 'CO2/12C-16O2/UCL-4000', 'H2O/1H2-16O/POKAZATEL', 'H3O_p/1H3-16O_p/eXeL', 'NH3/14N-1H3/CoYuTe', 'TiO/46Ti-16O/Toto', 'TiO/47Ti-16O/Toto', 'TiO/48Ti-16O/Toto', 'TiO/49Ti-16O/Toto', 'TiO/50Ti-16O/Toto']


In [ ]:
molparam_url = 'https://hitran.org/docs/iso-meta/'
hitran_reader = pd.read_html(molparam_url)

In [ ]:
hitran_reader[0]

In [ ]:
unc_formula = pd.DataFrame(eval(str(iso_slug_list).replace('1H','H').replace('-','').replace('_p','+')))
unc_formula.columns = ['exomol formula']
unc_formula

In [ ]:
hitran_online = pd.DataFrame()
hitran_online['exomol formula'] = unc_formula['exomol formula']
hitran_online['molecule id'] = pd.Series()
hitran_online['isotopologue id'] = pd.Series()
hitran_online['fractional abundance'] = pd.Series()
hitran_num = hitran_online['exomol formula'].count()

for k in range(hitran_num):
    for i in range(len(hitran_reader)):
        formula_n = hitran_reader[i]['Formula'].count()
        for j in range(formula_n):
            if (hitran_online['exomol formula'][k] == hitran_reader[i]['Formula'][j]):
                hitran_online['molecule id'][k] = int(i + 1)
                hitran_online['isotopologue id'][k] = int(j + 1)
                hitran_online['fractional abundance'][k] = hitran_reader[i]['Abundance'][j]


hitran_online['molecule id'] = [str(hitran_online['molecule id'][k]).replace('nan',str(49+k)) for k in range(hitran_num)]
hitran_online['isotopologue id'] = [str(hitran_online['isotopologue id'][k]).replace('nan','1') for k in range(hitran_num)]
hitran_online['fractional abundance'] = [str(hitran_online['fractional abundance'][k]).replace('nan','1') for k in range(hitran_num)]

hitran_online


Since the information is not complete and some of our considered molecules are not in the table of HITRAN online, we list them here for the following code. The information for those not included molecules are  hypothetic.

In [ ]:
'''
hitran_online = pd.DataFrame()
hitran_online['exomol formula'] = unc_formula['exomol formula']
hitran_online['molecule ID'] = ['50','26','51','2','1','52','11','53','53','53','53','53']
hitran_online['isotopologue ID'] = ['1','1','1','1','1','1','1','1','2','3','4','5']
hitran_online['fractional  abundance'] = ['1','0.977599','1','0.984204','0.997317','1','0.995872','0.2','0.2','0.2','0.2','0.2']
#hitran_online
'''

In [ ]:
'''
hitran_num = len(hitran_reader)
molecule_id = []
isotopologue_id = []
formulas = []
fractional_abundance = []

for formula in hitran_formula[0]:
    for i in range(hitran_num):
        formula_n = hitran_reader[i]['Formula'].count()
        for j in range(formula_n):
            if (formula == hitran_reader[i]['Formula'][j]):
                id_i = i
                id_j = j
                molecule_id.append(int(id_i + 1))
                isotopologue_id.append(int(id_j + 1))
                formulas.append(hitran_reader[i]['Formula'][j])
                fractional_abundance.append(hitran_reader[i]['Abundance'][j])

hitran_online = pd.DataFrame()
hitran_online['molecule ID'] = molecule_id
hitran_online['isotopologue ID'] = isotopologue_id
hitran_online['exomol formula'] = formulas
hitran_online['fractional abundance'] = fractional_abundance
hitran_online
'''

In [ ]:
# !!!!!!!

In [ ]:
hitran_online = pd.DataFrame()
hitran_online['molecule ID'] = ['11']
hitran_online['isotopologue ID'] = ['1']
hitran_online['exomol formula'] = ['14NH3']
hitran_online['fractional abundance'] = ['0.995872']
hitran_online

In [ ]:
#path_mol_iso_list = ['AlH/27Al-1H/AlHambra']

In [ ]:
path_mol_iso_list = ['NH3/14N-1H3/CoYuTe']

In [ ]:
T = 296                      # Reference temperature k
h = 6.62607015e-34           # Planck's const (J s)
c = 299792458                # Velocity of light (m s-1)
kB = 1.380649e-23            # Boltzmann's const (J K-1)

c2 = h * c * 100 / kB                  # Second radiation constant (cm K)
pi_c_8 = 1 / (8 * np.pi * c * 100)     # 8 * pi * c (cm-1 s)
c2_T = c2 / T                          # c2 / T  (cm)


In [ ]:
def extract_unc_trans(trans_df):
    upper_id = trans_df['i'].values
    lower_id = trans_df['f'].values
    state_id = unc_states_df['N'].values
    
    # Extract the same upper states ID from states_df
    unc_trans_i_df = pd.DataFrame()
    for id_1 in tqdm(state_id):
        unc_trans_i_df = unc_trans_i_df.append(trans_df[trans_df['i'].isin([id_1])])
        
    # Extract the same lower states ID from states_df
    unc_trans_df = pd.DataFrame()
    for id_2 in tqdm(state_id):
        unc_trans_df = unc_trans_df.append(unc_trans_i_df[unc_trans_i_df['f'].isin([id_2])])
        
    return unc_trans_df

In [ ]:
def calculate_csv(unc_states_df, unc_trans_df):
    

    unc_upper_id = unc_trans_df['i'].values
    unc_lower_id = unc_trans_df['f'].values 
    state_id = unc_states_df['N']
    unc_trans_num = unc_trans_df['i'].count()

    wavenumber = []                         # Vacuum wavenumber (cm−1)
    intensity = pd.DataFrame()              # Intensities (cm-1/molecule cm-2) at standard 296K         
    A_coefficient = []                      # Einstein A-coefficient
    lower_state_energy = pd.DataFrame()     # lower state energy
    uncertainty = []                        # Uncertainty indices
    weight_upper_state = pd.DataFrame()     # Nuclear-spin statistical weight of upper state
    weight_lower_state = pd.DataFrame()     # Nuclear-spin statistical weight of lower state


    for i in tqdm(range(unc_trans_num)):
        id_i = unc_upper_id[i]
        id_f = unc_lower_id[i]
        A = unc_trans_df['A_if'].values[i]                               # Einstein-A coefficient (s−1)
        J_i = unc_states_df[state_id.isin([id_i])]['J'].values           # Corresponding total angular momentum
        g_i_ns = unc_states_df[state_id.isin([id_i])]['g'].values        # Nuclear-spin statistical weight of upper state
        g_f_ns = unc_states_df[state_id.isin([id_f])]['g'].values        # Nuclear-spin statistical weight of lower state
        E_i = unc_states_df[state_id.isin([id_i])]['E'].values           # Upper state energy
        E_f = unc_states_df[state_id.isin([id_f])]['E'].values           # Lower state energy
        unc_i = unc_states_df[state_id.isin([id_i])]['Unc'].values       # Uncertainty indices of upper state
        unc_f = unc_states_df[state_id.isin([id_i])]['Unc'].values       # Uncertainty indices of lower state

        g_i_tot = g_i_ns * (2 * J_i + 1)                                 # Total degeneracy
        unc = math.sqrt(unc_i ** 2 + unc_f ** 2)                         # Uncertainty idices
        v = float(E_i - E_f)                                             # Vacuum wavenumber (cm−1)
        S = g_i_tot * A * np.exp(- c2_T * E_f) * (1 - np.exp(- c2_T * v)) * pi_c_8 / (v ** 2) / Q    # Intensities

        wavenumber.append(v)
        intensity = intensity.append(pd.DataFrame(S))
        A_coefficient.append(A)
        lower_state_energy = lower_state_energy.append(pd.DataFrame(E_f))
        uncertainty.append(unc)
        weight_upper_state = weight_upper_state.append(pd.DataFrame(g_i_ns))
        weight_lower_state = weight_lower_state.append(pd.DataFrame(g_f_ns))
    
    iso_csv_df = pd.DataFrame()
    iso_csv_df['v'] = wavenumber                       # Vacuum wavenumber (cm−1)
    iso_csv_df['S'] = intensity.values                 # Intensities (cm-1/molecule cm-2) at standard 296K  
    iso_csv_df['A'] = A_coefficient                    # Einstein A-coefficient
    iso_csv_df['E_f'] = lower_state_energy.values      # Lower state energy
    iso_csv_df['Ierr'] = uncertainty                   # Uncertainty indices
    iso_csv_df['g_i'] = weight_upper_state.values      # Nuclear-spin statistical weight of upper state
    iso_csv_df['g_f'] = weight_lower_state.values      # Nuclear-spin statistical weight of lower state
    iso_csv_df[M_mol_iso] = molecule_id                # Molecule number
    iso_csv_df['I'] = isotopologue_id                  # Isotopologue number
    iso_csv_df['gm_a'] = np.nan                        # Air-broadened half-width
    iso_csv_df['gm_s'] = np.nan                        # Self-broadened half-width
    iso_csv_df['n_a'] = np.nan                         # Temperature-dependence exponent for gamma_air
    iso_csv_df['dt_a'] = np.nan                        # Air pressure-induced line shift
    iso_csv_df['V_i'] = np.nan                         # Upper-state 'global' quanta
    iso_csv_df['V_f'] = np.nan                         # Lower-state 'global' quanta
    iso_csv_df['Q_i'] = np.nan                         # Upper-state 'local' quanta
    iso_csv_df['Q_f'] = np.nan                         # Lower-state 'local' quanta
    iso_csv_df['Iref'] = np.nan                        # Reference indices
    iso_csv_df['*'] = np.nan                           # Flag

    return iso_csv_df
    

In [ ]:
def convert_uncertainty_code(HITRAN_df):
    HITRAN_num = HITRAN_df['Ierr'].count()
    Ierr = []
    for i in range(HITRAN_num):
        uncertainty = HITRAN_df['Ierr'].values[i]
        uncertainty_value = float(uncertainty)
        if (0.0001 <= uncertainty_value < 0.001):
            uncertainty_code = '{:_>6}'.format(4)
        elif (0.00001 <= uncertainty_value < 0.0001):
            uncertainty_code = '{:_>6}'.format(5)
        elif (0.000001 <= uncertainty_value < 0.00001):
            uncertainty_code = '{:_>6}'.format(6)
        elif (0.0000001 <= uncertainty_value < 0.000001):
            uncertainty_code = '{:_>6}'.format(7)
        elif (uncertainty_value < 0.0000001):
            uncertainty_code = '{:_>6}'.format(8)
        Ierr.append(uncertainty_code)
    return Ierr

In [ ]:
def convert_csv_to_HITRAN(csv_df):
    HITRAN_df = csv_df[csv_df.S > 1.0E-30]
    Ierr = convert_uncertainty_code(HITRAN_df)
    
    HITRAN_df['M'] = HITRAN_df.M.map('{:_>2}'.format)
    HITRAN_df['I'] = HITRAN_df.I.map('{:>1}'.format)
    HITRAN_df['v'] = HITRAN_df.v
    HITRAN_df['S'] = HITRAN_df.S * fractional_abundance
    HITRAN_df['S'] = HITRAN_df.S.map('{:_>10.3E}'.format)
    HITRAN_df['A'] = HITRAN_df.A.map('{:_>10.3E}'.format)
    HITRAN_df['gm_a'] = '_' * 5
    HITRAN_df['gm_s'] = '_' * 5
    HITRAN_df['E_f'] = HITRAN_df.E_f.map('{:_>10.4F}'.format)
    HITRAN_df['n_a'] = '_' * 4
    HITRAN_df['dt_a'] = '_' * 8
    HITRAN_df['V_i'] = '_' * 15
    HITRAN_df['V_f'] = '_' * 15
    HITRAN_df['Q_i'] = '_' * 15
    HITRAN_df['Q_f'] = '_' * 15
    HITRAN_df['Ierr'] = Ierr
    HITRAN_df['Iref'] = '_' * 12
    HITRAN_df['*'] = '_'
    HITRAN_df['g_i'] = HITRAN_df.g_i.map('{:_>7.1F}'.format)
    HITRAN_df['g_f'] = HITRAN_df.g_f.map('{:_>7.1F}'.format)
    
    HITRAN_df = HITRAN_df.sort_values(['v'], ascending = True).reset_index(drop=True)
    HITRAN_df['v'] = HITRAN_df['v'].map('{:_>12.6F}'.format)
    return HITRAN_df


In [ ]:
read_path = './data/www.exomol.com/db/'

# Create a folder for saving result files.
# If the folder exists, delete (empty) the folder then create it.
result_path = './data/demo_result/'
if os.path.exists(result_path):                    # Determine whether the folder exists or not.
    for root, dirs, files in os.walk(result_path, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))    # Delete files in the folder.
        for name in dirs:
            os.rmdir(os.path.join(root, name))     # Delete the sub-folder.
    os.rmdir(result_path)                          # Delete the folder.
os.mkdir(result_path)                              # Create the folder.


In [ ]:
states_col_name = ['N','E','g','J','Unc']
trans_col_name = ['i', 'f', 'A_if']
pf_col_name = ['T', 'Q']

mol_iso_num = len(path_mol_iso_list)
for i in tqdm(range(mol_iso_num)):
    path_mol_iso = path_mol_iso_list[i]
    M_mol_iso = 'M of ' + path_mol_iso.replace('/','__')
    molecule_id = int(hitran_online['molecule ID'][i])
    isotopologue_id = int(hitran_online['isotopologue ID'][i])
    fractional_abundance = float(hitran_online['fractional abundance'][i])

    pf_url = ('http://www.exomol.com/db/' + path_mol_iso + '/'
              + path_mol_iso.split('/')[1] + '__'
              + path_mol_iso.split('/')[2] + '.pf')   
    response = requests.get(pf_url)
    content = response.text  
    pf_data = pd.read_csv(StringIO(content), sep='\s+', names=pf_col_name, header=None, engine='python')
    pf_path = read_path + path_mol_iso + '/' + path_mol_iso.replace('/','__') + '_pf.csv'
    pf_data.to_csv(pf_path, header=False)
    pf_df = pd.read_csv(pf_path, usecols=[0,1], header=None, names=pf_col_name)
    # Partition function defined as sum over states at standard 296K
    Q = pf_df['Q'].values[296-1]
        
    s_df = dict()
    states_df = pd.DataFrame()
    states_filenames = glob.glob(read_path + path_mol_iso + '/' + '*states.bz2')

    for states_filename in states_filenames:
        s_df[states_filename] = pd.read_csv(states_filename, compression='bz2', sep='\s+', usecols=[0,1,2,3,4], header=None,
                                            names=states_col_name, chunksize=100_000_000,
                                            iterator=True, low_memory=False)
        for chunk in s_df[states_filename]:
            states_df = states_df.append(chunk)

    # Extract rows of states file whose uncertainty indices are small than 0.001.
    unc_states_df = states_df[states_df['Unc'] < float(0.001)]
    
    t_df = dict()
    species_csv_df = pd.DataFrame()
    trans_filenames = glob.glob(read_path + path_mol_iso + '/' + '*trans.bz2')
    for trans_filename in tqdm(trans_filenames):
        t_df[trans_filename] = pd.read_csv(trans_filename, compression='bz2', sep='\s+', usecols=[0,1,2], header=None,
                                           names=trans_col_name, chunksize=100_000_000,
                                           iterator=True, low_memory=False)

        for trans_df in t_df[trans_filename]:
            unc_trans_df = extract_unc_trans(trans_df)
            if len(unc_trans_df) != 0:
                iso_csv_df = calculate_csv(unc_states_df, unc_trans_df)
            
        species_csv_df = species_csv_df.append(iso_csv_df)

    order = [M_mol_iso, 'I', 'v', 'S', 'A', 'gm_a', 'gm_s', 'E_f', 'n_a', 'dt_a',
                 'V_i', 'V_f', 'Q_i', 'Q_f', 'Ierr', 'Iref', '*', 'g_i', 'g_f']
    species_csv_df = species_csv_df[order]
    # Sort by increasing wavenumber
    species_csv_df = species_csv_df.sort_values(['v'], ascending = True).reset_index(drop=True)

    species_csv_df.to_csv(result_path + path_mol_iso.replace('/','__') + '.csv', header=True, index=False)   
    species_csv_df.to_csv(result_path + 'csv_result.csv', header=True, index=False, mode='a')
    

In [ ]:
species_csv_df

# Read result

In [ ]:
result_col_name = ['M', 'I', 'v', 'S', 'A', 'gamma_air', 'gamma_self',
                   'E_f', 'n_air', 'delta_air', 'V_i', 'V_f', 'Q_i', 'Q_f',
                   'Ierr', 'Iref', '*', 'g_i', 'g_f']

csv_reader = pd.read_csv(result_path + 'csv_result.csv', names = result_col_name, header=None, chunksize=100_000_000, iterator=True, low_memory=False)
for csv_df in csv_reader:
    csv_df = csv_df[~csv_df['I'].isin(['I'])]
    csv_df = csv_df.astype(float)
    HITRAN_df = convert_csv_to_HITRAN(csv_df)

In [ ]:
HITRAN_df

In [ ]:
HITRAN_df.to_csv(result_path + 'demo.txt', header=None, index=False)

In [ ]:
def read_txt_in_chunks(path, chunk_size=1024*1024):
    file = open(path, 'r')
    while True:
        chunk_data = file.read(chunk_size)
        if not chunk_data:
            break
        yield chunk_data

HITRAN_path = result_path + 'demo.txt'
with open(result_path + 'HITRAN_result.txt', 'w') as save_file:
    for chunk in read_txt_in_chunks(HITRAN_path):
        string = str(chunk).replace(',','').replace('_',' ')
        save_file.write(string)


In [ ]:
#test!!!!!!!!!!!!!!

In [ ]:
for i in tqdm(range(mol_iso_num)):
    path_mol_iso = path_mol_iso_list[i]
    def_path = glob.glob('./data/def/' + '*' + path_mol_iso.split('/')[1] + '__' + path_mol_iso.split('/')[2] + '.def')
    def_reader = pd.read_csv(def_path[0], sep='\\s+', names=['1','2','3','4','5'], header=None)

In [ ]:
def_reader[def_reader['4'].isin(['label'])]

In [ ]:
states_col_name = ['N','E','g','J','Unc'] + list(def_reader[def_reader['4'].isin(['label'])]['1'].values)

In [ ]:
states_col_name